In [ ]:
import argparse
import cv2

In [2]:
# Check video dimensions
cap = cv2.VideoCapture("../data/raw/GX010013.MP4")
ret, frame = cap.read()
if ret:
    h, w, _ = frame.shape
    print(f"Width: {w}, Height: {h}")
cap.release()



Width: 5312, Height: 2988


In [ ]:
# Select ROI interactively
cap = cv2.VideoCapture("../data/raw/GX010013.MP4")
ret, frame = cap.read()
cap.release()

if ret:
    cv2.namedWindow("Select ROI", cv2.WINDOW_NORMAL)
    cv2.resizeWindow("Select ROI", 1280, 720)
    roi = cv2.selectROI("Select ROI", frame, showCrosshair=True)
    cv2.destroyAllWindows()

    x, y, w, h = roi
    print(f"Selected ROI -> x:{x}, y:{y}, w:{w}, h:{h}")

Select a ROI and then press SPACE or ENTER button!
Cancel the selection process by pressing c button!
Selected ROI -> x:715, y:181, w:3079, h:1325


In [ ]:
# Display cropped frame for verification
cropped = frame[y:y+h, x:x+w]
cv2.imshow("Cropped", cropped)
cv2.waitKey(0)
cv2.destroyAllWindows()


In [3]:
# Crop video function
def crop_video(input_path, x, y, w, h, output_path=None):
    if output_path is None:
        output_path = input_path[:-4] + "_crop.MP4"

    cap = cv2.VideoCapture(input_path)
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    fourcc = cv2.VideoWriter_fourcc(*"mp4v")
    out = cv2.VideoWriter(output_path, fourcc, fps, (w, h))

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        cropped = frame[y:y+h, x:x+w]
        out.write(cropped)
    cap.release()
    out.release()

# ROI (Region of Interest) coordinates
X, Y, W, H = 715, 181, 3079, 1325  
crop_video("../data/raw/GX010013.MP4", X, Y, W, H)